## Model XGBoost - Aggression Data

In [1]:
#Imports
import pandas as pd
import numpy as np

from eda_functions import split_data

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import pickle

In [2]:
# Read in cleaned aggression file
aggression_df = pd.read_csv('../data/aggression_clean_data.csv')
aggression_df.head()

,rev_id,comment,year,logged_in,ns,sample,split,aggression,aggression_score,label
0,37675,This is not creative Those are the dictionary...,2002,True,article,random,train,0.100000,0.000000,0
1,44816,the term standard model is itself less NPOV t...,2002,True,article,random,train,0.000000,0.111111,0
2,49851,True or false the situation as of March 2002 w...,2002,True,article,random,train,0.000000,0.100000,0
3,89320,Next maybe you could work on being less conde...,2002,True,article,random,dev,0.444444,-0.444444,0
4,93890,This page will need disambiguation,2002,True,article,random,train,0.000000,0.333333,0


In [3]:
# Drop unnecessary columns and rename label column
aggression_df = aggression_df[['rev_id','comment','label']]
aggression_df.rename(columns={'label':'target'}, inplace=True)
aggression_df.head()

,rev_id,comment,target
0,37675,This is not creative Those are the dictionary...,0
1,44816,the term standard model is itself less NPOV t...,0
2,49851,True or false the situation as of March 2002 w...,0
3,89320,Next maybe you could work on being less conde...,0
4,93890,This page will need disambiguation,0


In [4]:
# Apply the custom train test split function to balance the classes in the training data only
X_train, X_test, y_train, y_test = split_data(
    aggression_df,
    pct_positive=0.5,
    random_state=42)

In [5]:
# Set up a pipeline with CountVectorizer and XGBClassifier
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('xgb', XGBClassifier(use_label_encoder=False, ))
])

In [6]:
# Set parameters
pipe_params = {
    'cvec__max_features': [5_000],
    'cvec__min_df'      : [3],
    'cvec__max_df'      : [.95],
    'cvec__stop_words'  : ['english'],
    'cvec__token_pattern' : [r'\w+|[A-Z]\w+'],
    'cvec__strip_accents' : ['ascii'],
    'cvec__ngram_range' : [(1,1)],
    'xgb__colsample_bytree': [0.75],
    'xgb__n_estimators': [250]
}

In [7]:
# Instatiate Gridsearch
gs = GridSearchCV(estimator = pipe,
                     param_grid = pipe_params,
                     cv = 5)

In [8]:
gs.fit(X_train, y_train)

[15:46:38] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:44] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:46:49] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('xgb',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                   

In [9]:
print(f'Train score: {gs.score(X_train, y_train)}')
print(f'Test score: {gs.score(X_test, y_test)}')

Train score: 0.8961971121558092
Test score: 0.8925448090743787


In [10]:
with open('../model_for_app/xgboost_aggression.pkl', 'wb') as pickle_out:
    pickle_out = pickle.dump(gs, pickle_out)